In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math
from secrets import IEX_CLOUD_API_TOKEN

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 79858214,
 'calculationPrice': 'tops',
 'change': 2.07,
 'changePercent': 0.01403,
 'close': 0,
 'closeSource': 'lfcoifai',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 149.3,
 'iexAskSize': 103,
 'iexBidPrice': 151.37,
 'iexBidSize': 206,
 'iexClose': 150.59,
 'iexCloseTime': 1689850964755,
 'iexLastUpdated': 1683667854401,
 'iexMarketPercent': 0.01194783355919346,
 'iexOpen': 152.4,
 'iexOpenTime': 1673131052154,
 'iexRealtimePrice': 151.49,
 'iexRealtimeSize': 104,
 'iexVolume': 529903,
 'lastTradeTime': 1666128078934,
 'latestPrice': 148.73,
 'latestSource': 'IEX real time price',
 'latestTime': '12:20:03 PM',
 'latestUpdate': 1708544109903,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'ma

In [4]:
pe_ratio = data['peRatio']
pe_ratio



30.53

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy


In [8]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series(
                                        [
                                        symbol,
                                        data[symbol]['quote']['latestPrice'],
                                        data[symbol]['quote']['peRatio'],
                                        'N/A'
                                        ],
                                        index = my_columns),
                                    ignore_index=True)
        
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,162.100,50.53,N/A
1,AAL,20.543,-2.0,N/A
2,AAP,233.250,24.3,N/A
3,AAPL,150.970,29.67,N/A
4,ABBV,112.750,30.37,N/A
...,...,...,...,...
500,YUM,128.240,30.29,N/A
501,ZBH,154.420,35.25,N/A
502,ZBRA,536.740,38.3,N/A
503,ZION,63.406,6.29,N/A


In [23]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,76.270,4.23,N/A
1,BIO,768.040,5.7,N/A
2,MPC,69.070,5.7,N/A
3,ZION,63.406,6.29,N/A
4,PRU,114.120,6.36,N/A
5,GM,57.964,6.66,N/A
6,CINF,119.940,6.78,N/A
7,AFL,58.315,6.92,N/A
8,COF,172.390,6.93,N/A
9,COO,405.512,6.94,N/A


In [24]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [25]:
portfolio_input()

Enter the value of your portfolio:1234567


In [28]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,76.270,4.23,323
1,BIO,768.040,5.7,32
2,MPC,69.070,5.7,357
3,ZION,63.406,6.29,389
4,PRU,114.120,6.36,216
5,GM,57.964,6.66,425
6,CINF,119.940,6.78,205
7,AFL,58.315,6.92,423
8,COF,172.390,6.93,143
9,COO,405.512,6.94,60
